In [7]:
import pandas as pd
import re
import numpy as np
from numpy.linalg import norm
from sklearn.feature_extraction.text import CountVectorizer
from scipy.stats import pearsonr
from scipy.spatial import distance

In [2]:
df = pd.read_csv('data.csv')

x = df['title']
x = [re.sub(r'[\W_]+', ' ', i.lower()) for i in x]
vectorizer = CountVectorizer()
vectorizer.fit_transform(x)

<90x77 sparse matrix of type '<class 'numpy.int64'>'
	with 268 stored elements in Compressed Sparse Row format>

In [8]:
def find_sim(search_key, data, method='cosine'):

    search_key = [search_key.lower()]
    key_vector = vectorizer.transform(search_key).toarray().flatten()
    result = {}

    if method == 'cosine':
        for i in data:
            tmp = re.sub(r'[\W_]+', ' ', i.lower())
            tmp = vectorizer.transform([i]).toarray().flatten()
            cosine = np.dot(key_vector, tmp)/(norm(key_vector)*norm(tmp))
            result[i] = cosine

    if method == 'euclidean':
        for i in data:
            tmp = re.sub(r'[\W_]+', ' ', i.lower())
            tmp = vectorizer.transform([i]).toarray().flatten()
            euclidean = norm(np.array(tmp) - np.array(key_vector))
            result[i] = euclidean

    if method == 'manhattan':
        for i in data:
            tmp = re.sub(r'[\W_]+', ' ', i.lower())
            tmp = vectorizer.transform([i]).toarray().flatten()
            manhattan = np.sum(np.absolute(tmp - key_vector))
            result[i] = manhattan / 100

    if method == 'pearson':
        for i in data:
            tmp = re.sub(r'[\W_]+', ' ', i.lower())
            tmp = vectorizer.transform([i]).toarray().flatten()
            pearson = 1 - pearsonr(key_vector, tmp)
            result[i] = pearson

    if method == 'chi2':
        for i in data:
            tmp = re.sub(r'[\W_]+', ' ', i.lower())
            tmp = vectorizer.transform([i]).toarray().flatten()
            chi2 = distance.chisquare(key_vector, tmp)
            result[i] = chi2

    result = dict(sorted(result.items(), key=lambda item: item[1])[::-1])
    return result

find_sim('web design', [i for i in df['title']], )